In [13]:
import scanpy as sc
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np


data_dir = "/Users/hyunjaecho/Downloads/raw_feature_bc_matrix/"

adata = sc.read_10x_mtx(data_dir, var_names='gene_symbols', cache=False)


In [18]:
b2type = pd.read_csv("/Users/hyunjaecho/Downloads/raw_feature_bc_matrix/barcode_to_celltype.tsv", sep="\t", header=None).iloc[1:]
b2type.columns = ["barcode", "celltype"]
b2type.head()

,barcode,celltype
1,AAACAGCCAAAGCCTC-1,Naive CD8+ T cells
2,AAACAGCCAGAATGAC-1,Classical Monocytes
3,AAACAGCCAGCTACGT-1,Classical Monocytes
4,AAACAGCCAGGCCTTG-1,Myeloid Dendritic cells
5,AAACAGCCATAATGTC-1,Naive CD4+ T cells


In [20]:
b2type.celltype.unique()

array(['Naive CD8+ T cells', 'Classical Monocytes',
       'Myeloid Dendritic cells', 'Naive CD4+ T cells',
       'CD8+ NKT-like cells', 'Effector CD4+ T cells', 'Pre-B cells',
       'Non-classical monocytes', 'Naive B cells', 'Plasma B cells',
       'Macrophages', 'Natural killer  cells',
       'Plasmacytoid Dendritic cells'], dtype=object)

In [22]:
# Find out overlapping barcodes from adata and b2type.
adata_barcodes = adata.obs.index
b2type_barcodes = b2type.barcode
overlapping_barcodes = np.intersect1d(adata_barcodes, b2type_barcodes)
len(overlapping_barcodes)

12313

In [27]:
adata[overlapping_barcodes,:].write_h5ad("/Users/hyunjaecho/Downloads/raw_feature_bc_matrix/pbmc10k.h5ad")

/Users/hyunjaecho/opt/anaconda3/envs/single_cell/lib/python3.8/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [28]:
# Read the h5ad file.
adata2 = sc.read_h5ad("/Users/hyunjaecho/Downloads/raw_feature_bc_matrix/pbmc10k.h5ad")

In [29]:
adata2

AnnData object with n_obs × n_vars = 12313 × 36601
    var: 'gene_ids', 'feature_types'

In [31]:
labels = b2type[b2type.barcode.isin(overlapping_barcodes)].celltype

In [33]:
labels

1             Naive CD8+ T cells
2            Classical Monocytes
3            Classical Monocytes
4        Myeloid Dendritic cells
5             Naive CD4+ T cells
                  ...           
12309         Naive CD4+ T cells
12310        Classical Monocytes
12311         Naive CD4+ T cells
12312         Naive CD8+ T cells
12313        Classical Monocytes
Name: celltype, Length: 12313, dtype: object

In [34]:
# If you want, you can also store the labels in the adata object:
adata2.obs['labels'] = labels